# Denoising EXR

In [15]:
import OpenEXR
import Imath
import numpy as np
import scipy.ndimage
import cv2
from skimage.restoration import denoise_nl_means

def read_exr(file_path):
    exr_file = OpenEXR.InputFile(file_path)
    header = exr_file.header()
    dw = header['dataWindow']
    size = (dw.max.x - dw.min.x + 1, dw.max.y - dw.min.y + 1)

    channels = {}
    for channel_name in exr_file.header()['channels'].keys():
        channel_data = exr_file.channel(channel_name, Imath.PixelType(Imath.PixelType.FLOAT))
        channel = np.frombuffer(channel_data, dtype=np.float32).reshape(size[1], size[0])
        channels[channel_name] = channel
    
    return channels, size

def write_exr(file_path, channels, size):
    header = OpenEXR.Header(size[0], size[1])
    for channel_name in channels.keys():
        header['channels'][channel_name] = Imath.Channel(Imath.PixelType(Imath.PixelType.FLOAT))
    
    exr_file = OpenEXR.OutputFile(file_path, header)
    channel_data = {name: data.tobytes() for name, data in channels.items()}
    exr_file.writePixels(channel_data)

patch_kw = dict(patch_size=5, patch_distance=6, channel_axis=-1)  # 5x5 patches  # 13x13 search area
def denoise_image(channels):
    denoised_channels = {}
    for channel_name, channel in channels.items():
        # denoised_channel = scipy.ndimage.median_filter(channel, size=8)
        # denoised_channel = scipy.ndimage.gaussian_filter(channel, sigma=1)
        print(channel.shape)
        denoised_channel = denoise_nl_means(channel, h=0.6 * 2, sigma=2, fast_mode=True, **patch_kw)
        denoised_channels[channel_name] = denoised_channel
    return denoised_channels

def main(input_file, output_file):
    channels, size = read_exr(input_file)
    denoised_channels = denoise_image(channels)
    write_exr(output_file, denoised_channels, size)


if __name__ == "__main__":
    input_file = "src/base_rafale.exr"
    output_file = "output/denoised_rafale.exr"
    main(input_file, output_file)


(1080, 1920)


NotImplementedError: Non-local means denoising is only implemented for 2D, 3D or 4D grayscale or multichannel images.

## Methode 2

In [4]:
import OpenEXR
import numpy as np
import Imath
import scipy.ndimage
import cv2

def load_exr(filename):
    file = OpenEXR.InputFile(filename)
    header = file.header()
    data_window = header["dataWindow"]
    height = data_window.max.y - data_window.min.y + 1
    width = data_window.max.x - data_window.min.x + 1

    channels = {}
    for channel_name in file.header()['channels'].keys():
        channel_data = file.channel(channel_name, Imath.PixelType(Imath.PixelType.FLOAT))
        channel = np.frombuffer(channel_data, dtype=np.float32).reshape(height, width)
        channels[channel_name] = channel

    return channels, data_window, header, height, width

channels, data_window, header, height, width = load_exr("src/base_rafale.exr")

In [5]:
channels

{'Ci.b': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'Ci.g': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'Ci.r': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'a': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0

In [113]:
data_window

(0, 0) - (1919, 1079)

In [12]:
header['channels']

{'Ci.b': HALF (1, 1),
 'Ci.g': HALF (1, 1),
 'Ci.r': HALF (1, 1),
 'a': HALF (1, 1),
 'albedo.b': HALF (1, 1),
 'albedo.g': HALF (1, 1),
 'albedo.r': HALF (1, 1),
 'albedo_mse.b': HALF (1, 1),
 'albedo_mse.g': HALF (1, 1),
 'albedo_mse.r': HALF (1, 1),
 'albedo_var.b': HALF (1, 1),
 'albedo_var.g': HALF (1, 1),
 'albedo_var.r': HALF (1, 1),
 'backward.x': HALF (1, 1),
 'backward.y': HALF (1, 1),
 'backward.z': HALF (1, 1),
 'diffuse.b': HALF (1, 1),
 'diffuse.g': HALF (1, 1),
 'diffuse.r': HALF (1, 1),
 'diffuse_mse.b': HALF (1, 1),
 'diffuse_mse.g': HALF (1, 1),
 'diffuse_mse.r': HALF (1, 1),
 'forward.x': HALF (1, 1),
 'forward.y': HALF (1, 1),
 'forward.z': HALF (1, 1),
 'mse.b': HALF (1, 1),
 'mse.g': HALF (1, 1),
 'mse.r': HALF (1, 1),
 'normal.x': HALF (1, 1),
 'normal.y': HALF (1, 1),
 'normal.z': HALF (1, 1),
 'normal_mse.x': HALF (1, 1),
 'normal_mse.y': HALF (1, 1),
 'normal_mse.z': HALF (1, 1),
 'normal_var.x': HALF (1, 1),
 'normal_var.y': HALF (1, 1),
 'normal_var.z': HALF

In [115]:
width, height

(1920, 1080)

In [6]:
channels["Ci.b"].shape

(1080, 1920)

In [164]:
import cv2


def denoise_nlm(image):
    image_uint8 = image.astype(np.uint32).astype(np.uint8)
    result = cv2.fastNlMeansDenoising(image_uint8, None, h=2, templateWindowSize=7, searchWindowSize=21)
    result_float = result.astype(np.float16)
    print(" -> ".join([str(image.sum()), str(image_uint8.sum()), str(result.sum()), str(result_float.sum())]))
    return result_float

result = {}
for ch, data in channels.items():
    print(ch)
    denoised_image = denoise_nlm(data)
    result[ch] = denoised_image

Ci.b
118795.21 -> 729 -> 0 -> 0.0
Ci.g
108075.11 -> 187 -> 0 -> 0.0
Ci.r
88775.45 -> 157 -> 0 -> 0.0
a
535381.3 -> 528017 -> 528825 -> inf
albedo.b
142809.23 -> 3 -> 0 -> 0.0
albedo.g
149420.52 -> 2 -> 0 -> 0.0
albedo.r
143678.78 -> 673 -> 0 -> 0.0
albedo_mse.b
31.685612 -> 0 -> 0 -> 0.0
albedo_mse.g
33.463387 -> 0 -> 0 -> 0.0
albedo_mse.r
32.7339 -> 0 -> 0 -> 0.0
albedo_var.b
505.08386 -> 0 -> 0 -> 0.0
albedo_var.g
533.36017 -> 0 -> 0 -> 0.0
albedo_var.r
521.84265 -> 0 -> 0 -> 0.0
backward.x
0.0 -> 0 -> 0 -> 0.0
backward.y
0.0 -> 0 -> 0 -> 0.0
backward.z
0.0 -> 0 -> 0 -> 0.0
diffuse.b
96356.23 -> 111 -> 0 -> 0.0
diffuse.g
88110.055 -> 35 -> 0 -> 0.0
diffuse.r
71530.0 -> 84 -> 0 -> 0.0
diffuse_mse.b
1088.2649 -> 0 -> 0 -> 0.0
diffuse_mse.g
894.6686 -> 0 -> 0 -> 0.0
diffuse_mse.r
599.2405 -> 0 -> 0 -> 0.0
forward.x
0.0 -> 0 -> 0 -> 0.0
forward.y
0.0 -> 0 -> 0 -> 0.0
forward.z
0.0 -> 0 -> 0 -> 0.0
mse.b
1497.2594 -> 0 -> 0 -> 0.0
mse.g
1197.6354 -> 0 -> 0 -> 0.0
mse.r
822.20996 -> 0 -> 0

In [161]:
channels["Ci.b"].sum()

118795.21

In [162]:
result["Ci.b"].sum()

0.0

In [163]:
def save_exr(filename, data, header):
    output = OpenEXR.OutputFile(filename, header)
    output.writePixels(data)
    output.close()


save_exr("output/denoised_rafale.exr", result, header)

In [7]:
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio
from skimage.util import random_noise

In [11]:
def denoise_nlm(image):
    print(type(image))
    patch_kw = dict(patch_size=5, patch_distance=6, channel_axis=-1)  # 5x5 patches  # 13x13 search area
    return denoise_nl_means(image, h=0.6 * 2, sigma=2, fast_mode=True, **patch_kw)

result = {}
for ch, data in channels.items():
    denoised_image = denoise_nlm(data)
    result[ch] = denoised_image

AttributeError: 'str' object has no attribute 'shape'